In [1]:
!pip install -q langchain
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00


In [3]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-'

In [5]:
import numpy as np
import pandas as pd
import sqlite3

df = pd.read_csv("data.csv")

df.rename(columns={'User ID': 'user_id',
                   'Product ID': 'product_id',
                   'Product Name' : 'product_name',
                   'Brand' : 'brand',
                   'Category' : 'category',
                   'Price' : 'price',
                   'Color' : 'color',
                   'Size' : 'size'}, inplace=True)

conn = sqlite3.connect('fashion_db.sqlite')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS fashion_products (user_id int, product_id int, product_name text, brand text, category text, price int, rating float, color text, size text)')
conn.commit()

df.to_sql('fashion_products', conn, if_exists='replace', index = False)

c.execute('''
SELECT product_name FROM fashion_products LIMIT 100
          ''')

for row in c.fetchall():
    print (row)


Reading the DB

In [ ]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()


In [ ]:
import sqlite3
read_sql_query('SELECT * FROM fashion_products LIMIT 10;',
               "fashion_db.sqlite")

LangChain Agent

In [ ]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
input_db = SQLDatabase.from_uri('sqlite:///fashion_db.sqlite')
llm_1 = OpenAI(temperature=0)
db_agent = SQLDatabaseChain(llm = llm_1,
                            database = input_db,
                            verbose=True)

In [ ]:
db_agent.run("how many rows are there?")


In [ ]:
db_agent.run("how many entries of Adidas are present?")


In [ ]:
db_agent.run("how many XL products of Nike are there that have a rating of more than 4?")


In [ ]:
db_agent.run("Give all the details of Adidas which have a size of L and have a rating of more than 4.2")
